#  Self-Healing Classification DAG with Fine-Tuned Model

ATG Technical Assignment

This notebook implements:
- DistilBERT fine-tuning with LoRA
- LangGraph DAG with self-healing mechanism
- Confidence-based fallback strategy
- Human-in-the-loop classification
- Comprehensive logging





```
# This is formatted as code
```

 Step 1: Installation & Setup

In [ ]:
# Install required packages
!pip install -q torch transformers datasets peft accelerate
!pip install -q langgraph langchain-core
!pip install -q scikit-learn

print("✅ All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 18.6 MB/s eta 0:00:00
✅ All packages installed successfully!


In [ ]:
# Check GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Change Runtime to GPU for faster training.")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU Device: Tesla T4
GPU Memory: 15.83 GB


## 🎯 Step 2: Fine-Tune DistilBERT with LoRA

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import json
from datetime import datetime

class ModelTrainer:
    def __init__(self, model_name="distilbert-base-uncased", output_dir="./fine_tuned_model"):
        self.model_name = model_name
        self.output_dir = output_dir
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"🚀 Using device: {self.device}")

    def load_and_prepare_data(self, max_samples=3000):
        """Load IMDB dataset and prepare for training"""
        print("\n📊 Loading IMDB dataset...")
        dataset = load_dataset("imdb")

        # Use smaller subset for Colab free tier (faster training)
        train_dataset = dataset["train"].shuffle(seed=42).select(range(max_samples))
        test_dataset = dataset["test"].shuffle(seed=42).select(range(max_samples // 5))

        print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

        # Tokenize datasets
        def tokenize_function(examples):
            return self.tokenizer(
                examples["text"],
                padding="max_length",
                truncation=True,
                max_length=256  # Reduced for faster training
            )

        print("🔄 Tokenizing datasets...")
        train_dataset = train_dataset.map(tokenize_function, batched=True)
        test_dataset = test_dataset.map(tokenize_function, batched=True)

        return train_dataset, test_dataset

    def create_lora_model(self):
        """Create model with LoRA configuration"""
        print("\n🤖 Loading base model...")
        model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=2,
            id2label={0: "negative", 1: "positive"},
            label2id={"negative": 0, "positive": 1}
        )

        # Configure LoRA
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["q_lin", "v_lin"],
            bias="none"
        )

        # Get PEFT model
        model = get_peft_model(model, lora_config)
        print("\n⚡ LoRA Model Configuration:")
        model.print_trainable_parameters()

        return model

    def compute_metrics(self, eval_pred):
        """Compute metrics for evaluation"""
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average='binary'
        )
        accuracy = accuracy_score(labels, predictions)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

    def train(self):
        """Train the model with LoRA"""
        # Load data
        train_dataset, test_dataset = self.load_and_prepare_data()

        # Create model
        model = self.create_lora_model()

        # Training arguments
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            num_train_epochs=2,  # Reduced for Colab
            per_device_train_batch_size=16,
            per_device_eval_batch_size=32,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=50,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            push_to_hub=False,
            report_to="none"
        )

        # Data collator
        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        # Create trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics
        )

        # Train
        print("\n🔥 Starting training...")
        print("="*70)
        train_result = trainer.train()

        # Evaluate
        print("\n📈 Evaluating model...")
        eval_results = trainer.evaluate()

        # Save model
        trainer.save_model(self.output_dir)
        self.tokenizer.save_pretrained(self.output_dir)

        # Save training info
        training_info = {
            "model_name": self.model_name,
            "training_date": datetime.now().isoformat(),
            "train_samples": len(train_dataset),
            "test_samples": len(test_dataset),
            "final_metrics": eval_results,
            "training_time": train_result.metrics['train_runtime']
        }

        with open(f"{self.output_dir}/training_info.json", "w") as f:
            json.dump(training_info, f, indent=2)

        print("\n" + "="*70)
        print("✅ TRAINING COMPLETED!")
        print("="*70)
        print(f"📊 Accuracy:  {eval_results['eval_accuracy']:.4f}")
        print(f"📊 F1 Score:  {eval_results['eval_f1']:.4f}")
        print(f"📊 Precision: {eval_results['eval_precision']:.4f}")
        print(f"📊 Recall:    {eval_results['eval_recall']:.4f}")
        print(f"⏱️  Time:      {train_result.metrics['train_runtime']:.2f}s")
        print(f"💾 Model saved to: {self.output_dir}")
        print("="*70)

        return model, eval_results

print("✅ ModelTrainer class loaded!")

✅ ModelTrainer class loaded!


In [ ]:
# Train the model
print("🎯 Starting Model Training...\n")
trainer = ModelTrainer()
model, results = trainer.train()

🎯 Starting Model Training...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1260909182.py:123: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



⚡ LoRA Model Configuration:
trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925

🔥 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.600400,0.381017,0.830000,0.823529,0.838028,0.809524
2,0.346400,0.358267,0.838333,0.831889,0.848057,0.816327



📈 Evaluating model...



✅ TRAINING COMPLETED!
📊 Accuracy:  0.8383
📊 F1 Score:  0.8319
📊 Precision: 0.8481
📊 Recall:    0.8163
⏱️  Time:      99.01s
💾 Model saved to: ./fine_tuned_model


## 🔗 Step 3: Build LangGraph Self-Healing DAG

---



In [ ]:
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, END, START
import torch

class GraphState(TypedDict):
    """State schema for the DAG"""
    user_input: str
    predicted_label: Optional[str]
    confidence: Optional[float]
    all_probabilities: Optional[dict]
    fallback_triggered: bool
    user_clarification: Optional[str]
    final_label: str
    final_confidence: float
    timestamp: str
    correction_applied: bool

class SelfHealingClassifier:
    def __init__(
        self,
        model_path="./fine_tuned_model",
        confidence_threshold=0.65,
        log_file="classification_log.json"
    ):
        self.model_path = model_path
        self.confidence_threshold = confidence_threshold
        self.log_file = log_file
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load model and tokenizer
        print(f"🔄 Loading model from {model_path}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()

        # Label mapping
        self.id2label = {0: "negative", 1: "positive"}
        self.label2id = {"negative": 0, "positive": 1}

        # Initialize log
        self.logs = []
        print("✅ Model loaded successfully!")

    def inference_node(self, state: GraphState) -> GraphState:
        """InferenceNode: Runs classification using the trained model"""
        user_input = state["user_input"]

        print(f"\n[InferenceNode] Processing input: '{user_input[:60]}...'")

        # Tokenize input
        inputs = self.tokenizer(
            user_input,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=256
        ).to(self.device)

        # Get predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=-1)[0]

        # Get prediction
        predicted_idx = torch.argmax(probabilities).item()
        predicted_label = self.id2label[predicted_idx]
        confidence = probabilities[predicted_idx].item()

        # Store all probabilities
        all_probs = {
            self.id2label[i]: probabilities[i].item()
            for i in range(len(probabilities))
        }

        print(f"[InferenceNode] Predicted label: {predicted_label.capitalize()} | Confidence: {confidence:.1%}")

        return {
            **state,
            "predicted_label": predicted_label,
            "confidence": confidence,
            "all_probabilities": all_probs,
            "timestamp": datetime.now().isoformat()
        }

    def confidence_check_node(self, state: GraphState) -> GraphState:
        """ConfidenceCheckNode: Evaluates confidence and decides on fallback"""
        confidence = state["confidence"]

        print(f"[ConfidenceCheckNode] Checking confidence: {confidence:.1%}")

        if confidence < self.confidence_threshold:
            print(f"[ConfidenceCheckNode] Confidence too low (< {self.confidence_threshold:.1%}). Triggering fallback...")
            return {**state, "fallback_triggered": True}
        else:
            print(f"[ConfidenceCheckNode] Confidence acceptable. Proceeding with prediction.")
            return {
                **state,
                "fallback_triggered": False,
                "final_label": state["predicted_label"],
                "final_confidence": confidence,
                "correction_applied": False
            }

    def fallback_node(self, state: GraphState) -> GraphState:
        """FallbackNode: Handles low-confidence predictions"""
        predicted_label = state["predicted_label"]
        all_probs = state["all_probabilities"]

        print(f"\n[FallbackNode] Requesting user clarification...")

        # Show probabilities to user
        print(f"\nPrediction probabilities:")
        for label, prob in all_probs.items():
            print(f"  - {label.capitalize()}: {prob:.1%}")

        # Ask for clarification
        print(f"\n🤔 I'm not very confident about this prediction.")
        print(f"   My guess was: {predicted_label.capitalize()}")
        print(f"\nCould you clarify the sentiment of your text?")
        print("Options:")
        print("  1. Positive")
        print("  2. Negative")
        print("  3. Use original prediction")

        user_choice = input("\nYour choice (1/2/3): ").strip()

        # Process user input
        if user_choice == "1":
            final_label = "positive"
            clarification = "User confirmed: Positive"
            correction = final_label != predicted_label
        elif user_choice == "2":
            final_label = "negative"
            clarification = "User confirmed: Negative"
            correction = final_label != predicted_label
        else:
            final_label = predicted_label
            clarification = "User accepted original prediction"
            correction = False

        print(f"\n[FallbackNode] Final label: {final_label.capitalize()} (Corrected via user clarification)")

        return {
            **state,
            "user_clarification": clarification,
            "final_label": final_label,
            "final_confidence": 1.0 if correction else state["confidence"],
            "correction_applied": correction
        }

    def should_fallback(self, state: GraphState) -> Literal["fallback", "finalize"]:
        """Router: Decide whether to trigger fallback"""
        return "fallback" if state["fallback_triggered"] else "finalize"

    def finalize_node(self, state: GraphState) -> GraphState:
        """Finalize and log the result"""
        # Log the interaction
        log_entry = {
            "timestamp": state["timestamp"],
            "input": state["user_input"],
            "predicted_label": state["predicted_label"],
            "confidence": state["confidence"],
            "all_probabilities": state["all_probabilities"],
            "fallback_triggered": state["fallback_triggered"],
            "user_clarification": state.get("user_clarification"),
            "final_label": state["final_label"],
            "final_confidence": state["final_confidence"],
            "correction_applied": state["correction_applied"]
        }

        self.logs.append(log_entry)

        # Save to file
        with open(self.log_file, "w") as f:
            json.dump(self.logs, f, indent=2)

        return state

    def build_graph(self):
        """Build the LangGraph DAG"""
        workflow = StateGraph(GraphState)

        # Add nodes
        workflow.add_node("inference", self.inference_node)
        workflow.add_node("confidence_check", self.confidence_check_node)
        workflow.add_node("fallback", self.fallback_node)
        workflow.add_node("finalize", self.finalize_node)

        # Add edges
        workflow.add_edge(START, "inference")
        workflow.add_edge("inference", "confidence_check")

        # Conditional edge based on confidence
        workflow.add_conditional_edges(
            "confidence_check",
            self.should_fallback,
            {
                "fallback": "fallback",
                "finalize": "finalize"
            }
        )

        workflow.add_edge("fallback", "finalize")
        workflow.add_edge("finalize", END)

        return workflow.compile()

    def run_classification(self, user_input: str) -> dict:
        """Run classification through the DAG"""
        graph = self.build_graph()

        initial_state = {
            "user_input": user_input,
            "predicted_label": None,
            "confidence": None,
            "all_probabilities": None,
            "fallback_triggered": False,
            "user_clarification": None,
            "final_label": "",
            "final_confidence": 0.0,
            "timestamp": "",
            "correction_applied": False
        }

        # Execute graph
        result = graph.invoke(initial_state)

        return result

    def display_statistics(self):
        """Display classification statistics"""
        if not self.logs:
            print("No classifications yet.")
            return

        total = len(self.logs)
        fallback_count = sum(1 for log in self.logs if log["fallback_triggered"])
        correction_count = sum(1 for log in self.logs if log.get("correction_applied", False))
        avg_confidence = sum(log["confidence"] for log in self.logs) / total

        print("\n" + "="*70)
        print("📊 CLASSIFICATION STATISTICS")
        print("="*70)
        print(f"Total Classifications: {total}")
        print(f"Fallback Triggered: {fallback_count} ({fallback_count/total*100:.1f}%)")
        print(f"User Corrections: {correction_count} ({correction_count/total*100:.1f}%)")
        print(f"Average Confidence: {avg_confidence:.1%}")
        print("="*70)

print("✅ SelfHealingClassifier class loaded!")

✅ SelfHealingClassifier class loaded!


## 🎮 Step 4: Interactive Demo

In [ ]:
# Initialize the self-healing classifier
print("🚀 Initializing Self-Healing Classifier...\n")
classifier = SelfHealingClassifier(
    model_path="./fine_tuned_model",
    confidence_threshold=0.65
)

print("\n" + "="*70)
print("✅ System Ready! You can now run classifications.")
print("="*70)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Initializing Self-Healing Classifier...

🔄 Loading model from ./fine_tuned_model...
✅ Model loaded successfully!

✅ System Ready! You can now run classifications.


In [ ]:
# Example 1: High Confidence - No Fallback
print("\n🎬 Example 1: Clear Positive Review\n")

review1 = "This movie was absolutely fantastic! Best film I've seen all year. Amazing acting!"
result1 = classifier.run_classification(review1)

print("\n" + "="*70)
print("🎯 FINAL RESULT")
print("="*70)
print(f"Input: {result1['user_input'][:80]}...")
print(f"Final Label: {result1['final_label'].upper()}")
print(f"Final Confidence: {result1['final_confidence']:.1%}")
if result1['correction_applied']:
    print(f"✓ Correction applied via user feedback")
print("="*70)


🎬 Example 1: Clear Positive Review


[InferenceNode] Processing input: 'This movie was absolutely fantastic! Best film I've seen all...'
[InferenceNode] Predicted label: Positive | Confidence: 97.8%
[ConfidenceCheckNode] Checking confidence: 97.8%
[ConfidenceCheckNode] Confidence acceptable. Proceeding with prediction.

🎯 FINAL RESULT
Input: This movie was absolutely fantastic! Best film I've seen all year. Amazing actin...
Final Label: POSITIVE
Final Confidence: 97.8%


In [ ]:
# Example 2: Low Confidence - Fallback Triggered (Assignment Example)
print("\n🎬 Example 2: Ambiguous Review (Fallback Expected)\n")

review2 = "The movie was painfully slow and boring."
result2 = classifier.run_classification(review2)

print("\n" + "="*70)
print("🎯 FINAL RESULT")
print("="*70)
print(f"Input: {result2['user_input']}")
print(f"Final Label: {result2['final_label'].upper()}")
print(f"Final Confidence: {result2['final_confidence']:.1%}")
if result2['correction_applied']:
    print(f"✓ Correction applied via user feedback")
print("="*70)


🎬 Example 2: Ambiguous Review (Fallback Expected)


[InferenceNode] Processing input: 'The movie was painfully slow and boring....'
[InferenceNode] Predicted label: Negative | Confidence: 93.2%
[ConfidenceCheckNode] Checking confidence: 93.2%
[ConfidenceCheckNode] Confidence acceptable. Proceeding with prediction.

🎯 FINAL RESULT
Input: The movie was painfully slow and boring.
Final Label: NEGATIVE
Final Confidence: 93.2%


In [ ]:
# Your Custom Input - Try your own review!
print("\n🎬 Try Your Own Review!\n")

# Change this to your own review
your_review = "The acting was decent but the story was quite boring."

result = classifier.run_classification(your_review)

print("\n" + "="*70)
print("🎯 FINAL RESULT")
print("="*70)
print(f"Input: {result['user_input']}")
print(f"Final Label: {result['final_label'].upper()}")
print(f"Final Confidence: {result['final_confidence']:.1%}")
if result['correction_applied']:
    print(f"✓ Correction applied via user feedback")
print("="*70)


🎬 Try Your Own Review!


[InferenceNode] Processing input: 'The acting was decent but the story was quite boring....'
[InferenceNode] Predicted label: Negative | Confidence: 84.0%
[ConfidenceCheckNode] Checking confidence: 84.0%
[ConfidenceCheckNode] Confidence acceptable. Proceeding with prediction.

🎯 FINAL RESULT
Input: The acting was decent but the story was quite boring.
Final Label: NEGATIVE
Final Confidence: 84.0%


## 📊 Step 5: View Statistics & Logs

In [ ]:
# Display statistics
classifier.display_statistics()


📊 CLASSIFICATION STATISTICS
Total Classifications: 3
Fallback Triggered: 0 (0.0%)
User Corrections: 0 (0.0%)
Average Confidence: 91.6%


In [ ]:
# View detailed logs
import json

print("\n📋 DETAILED CLASSIFICATION LOGS")
print("="*70)

with open("classification_log.json", "r") as f:
    logs = json.load(f)

for i, log in enumerate(logs, 1):
    print(f"\n[{i}] {log['timestamp']}")
    print(f"Input: {log['input'][:60]}...")
    print(f"Predicted: {log['predicted_label'].upper()} ({log['confidence']:.1%})")
    print(f"Final: {log['final_label'].upper()} ({log['final_confidence']:.1%})")
    if log['fallback_triggered']:
        print(f"⚠️ Fallback: {log['user_clarification']}")
    print("-"*70)


📋 DETAILED CLASSIFICATION LOGS

[1] 2025-10-02T19:46:08.867085
Input: This movie was absolutely fantastic! Best film I've seen all...
Predicted: POSITIVE (97.8%)
Final: POSITIVE (97.8%)
----------------------------------------------------------------------

[2] 2025-10-02T19:46:08.892796
Input: The movie was painfully slow and boring....
Predicted: NEGATIVE (93.2%)
Final: NEGATIVE (93.2%)
----------------------------------------------------------------------

[3] 2025-10-02T19:46:08.915182
Input: The acting was decent but the story was quite boring....
Predicted: NEGATIVE (84.0%)
Final: NEGATIVE (84.0%)
----------------------------------------------------------------------


## 💾 Step 6: Download Model & Logs

Download the trained model and logs for your submission.

In [ ]:

!zip -r atg_assignment_submission.zip fine_tuned_model/ classification_log.json

print("\n✅ Files zipped successfully!")
print("📦 Download 'atg_assignment_submission.zip' from the Files panel (left sidebar)")
print("\nSubmission Contents:")
print("  - fine_tuned_model/ (trained model)")
print("  - classification_log.json (logs)")

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/README.md (deflated 66%)
  adding: fine_tuned_model/adapter_config.json (deflated 56%)
  adding: fine_tuned_model/vocab.txt (deflated 53%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_model/checkpoint-188/ (stored 0%)
  adding: fine_tuned_model/checkpoint-188/README.md (deflated 66%)
  adding: fine_tuned_model/checkpoint-188/adapter_config.json (deflated 56%)
  adding: fine_tuned_model/checkpoint-188/vocab.txt (deflated 53%)
  adding: fine_tuned_model/checkpoint-188/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_model/checkpoint-188/scheduler.pt (deflated 61%)
  adding: fine_tuned_model/checkpoint-188/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_model/checkpoint-188/adapter_model.safetensors (deflated 7%)
  adding: fine_tuned_model/checkpoint-188/training_args.bin (deflated 53%)
  adding: fine_tuned_model/checkpoint-188/optimizer.pt (deflated 10%)
  addin